In [4]:
import sys
import os
import random
import numpy as np
from collections import Counter

class HybridRecSys:
    def __init__(self):
        self.id_user = dict()
        self.user_id = dict()
        self.id_game = dict()
        self.game_id = dict()
        self.id_item = dict()
        self.item_id = dict()
        self.item_user = dict()
        self.user_item = dict()
        self.game_user = dict()
        self.user_game = dict()
        self.game_item = dict()
        self.item_game = dict()
        self.test2days_item_user = dict()
        self.test2days_user_item = dict()
        self.test4days_item_user = dict()
        self.test4days_user_item = dict()
        self.test10days_item_user = dict()
        self.test10days_user_item = dict()
        self.user_friend = dict()
        self.user_recitems = dict()
        self.nuser_recitems = dict()
        self.userlist = []
        self.selected_user_items = dict()

    def read_data(self, dirpath):
        with open(dirpath + "/id_user.txt") as udictfile:
            for line in udictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_user[toks[0]] = toks[1].replace(" ", "")    # {uid1: User1, uid2: User2....}
                    self.user_id[toks[1]] = toks[0].replace(" ", "")    # {User1: uid1, User2: uid2....}

        print("#users", len(self.id_user))

        with open(dirpath + "/id_game.txt") as gdictfile:
            for line in gdictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_game[toks[0]] = toks[1].replace(" ", "")    # {33302: LoLsg, 33305: LoLmy....}
                    self.game_id[toks[1]] = toks[0].replace(" ", "")    # {LoLsg: 33302, LoLmy: 33305....}

        print("#games", len(self.id_game))
        
        with open(dirpath + "/id_item.txt") as idictfile:
            for line in idictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_item[toks[0]] = toks[1].replace(" ", "")    # {itemid1: Item1, itemid2: Item2....}
                    self.item_id[toks[1]] = toks[0].replace(" ", "")    # {Item1: itemid1, Item2: itemid2....}

        print("#items", len(self.id_item)) # comprehensive list of items

        with open(dirpath + "/game_user.txt") as gufile:  # data pulled from user game activity for 20 days
            for line in gufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    g, u = toks[0], toks[1]
                    if g not in self.game_user:
                        self.game_user[g] = []
                    self.game_user[g].append(u)   # {gameid1: [uid1,uid2,...], gameid2: [uid2,uid3,....]} = dict of list of users playing games
                    if u not in self.user_game:
                        self.user_game[u] = []
                    self.user_game[u].append(g)   # {uid1: [gameid1,gameid2,...], uid2: [gameid2,gameid3,....]} = dict of list of games played by each user
        sample_items = {k: self.user_game[k] for k in list(self.user_game)[:20]}
        print(sample_items)
        print("\n")
        
        
        with open(dirpath + "/item_user.txt") as iufile:  # list of all items purchased by users
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.item_user:
                        self.item_user[i] = []
                    self.item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.user_item:
                        self.user_item[u] = []
                    self.user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        sample_items = {k: self.user_item[k] for k in list(self.user_item)[:20]}
        print(sample_items)
        print("\n")
        
        
        with open(dirpath + "/item_game.txt") as igfile:  # list of all in-game items
            for line in igfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, g = toks[0], toks[1]
                    if i not in self.item_game:
                        self.item_game[i] = []  
                    self.item_game[i].append(g) # {itemid1: [gameid1,gameid2,...], itemid2: [gameid2,gameid3,....]} = list of items that got recommended. Every item can be in multiple games
                    if g not in self.game_item:
                        self.game_item[g] = []
                    self.game_item[g].append(i) # {gameid1: [itemid1,itemid2,...], gameid2: [itemd2,itemd3,....]} = list of items available in-game for purchase
        sample_items = {k: self.item_game[k] for k in list(self.item_game)[:20]}
        print(sample_items)
        print("\n")
        
        with open(dirpath + "/test2days_item_user.txt") as iufile:  # list of all items purchased by users in 2 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test2days_item_user:
                        self.test2days_item_user[i] = []
                    self.test2days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test2days_user_item:
                        self.test2days_user_item[u] = []
                    self.test2days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 2 days of test period:", len(self.test2days_user_item))
        
        with open(dirpath + "/test4days_item_user.txt") as iufile:  # list of all items purchased by users in 4 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test4days_item_user:
                        self.test4days_item_user[i] = []
                    self.test4days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test4days_user_item:
                        self.test4days_user_item[u] = []
                    self.test4days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 4 days of test period:", len(self.test4days_user_item))
        
        with open(dirpath + "/test10days_item_user.txt") as iufile:  # list of all items purchased by users in 10 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test10days_item_user:
                        self.test10days_item_user[i] = []
                    self.test10days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test10days_user_item:
                        self.test10days_user_item[u] = []
                    self.test10days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 10 days of test period:", len(self.test10days_user_item))
        
        with open(dirpath + "/selected_user.txt") as sufile:  # pull the randomly selected users
            for user in sufile:
                self.userlist.append(user.strip())
                
        with open(dirpath + "/user_friend.txt") as uffile:  # list of all friendship linkages
            for line in uffile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    u, f = toks[0], toks[1]
                    if u not in self.user_friend:
                        self.user_friend[u] = []
                    self.user_friend[u].append(f)   # {uid1: [friendid1,friendid2,...], uid2: [friendid2,friendid3,....]} = dict of each person's friends
        
    def read_meta2vec_output(self, outputfilename):
        self.user_vec = dict()
        self.game_vec = dict()
        self.item_vec = dict()
        print("\nmeta2vec User Vector Representations\n")
        with open(outputfilename) as nvfile:             
            for line in nvfile:
                toks = line.strip().split(" ")
                if len(toks) == 129:
                    node = toks[0]
                    del toks[0]
                    if node[0] == 'u': 
                        self.user_vec[self.user_id[node]] = toks    # {uid1: [user_vector_values], uid2: [user_vector_values]..}
                    elif node[0] == 'g':
                        self.game_vec[self.game_id[node]] = toks    # {gameid1: [game_vector_values], gameid2: [game_vector_values]..}
                    elif node[0] == 'i':
                        self.item_vec[self.item_id[node]] = toks    # {itemid1: [item_vector_values], itemid2: [item_vector_values]..}
        sample_items = {k: self.user_vec[k] for k in list(self.user_vec)[:2]}
        print(sample_items)
        print("\n")
        
    def read_LINE_output(self, outputfilename):
        self.user_line_vec = dict()
        print("\nLINE User Vector Representation\n")
        with open(outputfilename) as nvfile:             
            for line in nvfile:
                toks = line.strip().split(" ")
                if len(toks) == 129:
                    node = toks[0]
                    del toks[0] 
                    self.user_line_vec[node] = toks    # {uid1: [user_vector_values], uid2: [user_vector_values]..}
        sample_items = {k: self.user_line_vec[k] for k in list(self.user_line_vec)[:2]}
        print(sample_items)
        print("\n")
        
    def get_cosine_similarity(self, vec1, vec2):
        a = np.array(vec1, dtype=float)
        b = np.array(vec2, dtype=float)
        dot_product = np.dot(a, b)
        norm_a = np.linalg.norm(a)
        norm_b = np.linalg.norm(b)
        return dot_product / (norm_a * norm_b)
    
    def make_item_recommendations_selected_users(self):
        self.first_user_recitems = dict()
        self.second_user_recitems = dict()
        default_weight_meta = 0.43
        default_weight_line = 0.38
        default_weight_friends = 0.19
        for user1 in self.userlist:                                                                                # iterating through 10,000 randomly selected users
            user1_metavec = self.user_vec.get(user1,[])                                                            # getting the latent vector representation
            user1_linevec = self.user_line_vec.get(user1,[])
            if (user1_metavec != [] and user1_linevec !=[]):
                self.meta_user_simscore = {}
                self.line_user_simscore = {}
                self.user_simscore = {}
                self.user_rec = {}
                items1 = self.user_item.get(user1,[])                                                               # getting the items purchased
                friends1 = set(self.user_friend.get(user1,[]))
                for user2 in self.user_item:                                                                        # iterating through other users in the gaming network
                    user2_metavec = self.user_vec.get(user2,[])                                                     # getting the latent vector representation
                    user2_linevec = self.user_line_vec.get(user2,[])
                    if (user2_metavec != [] and user2_linevec !=[]):
                        items2 = self.user_item[user2]                                                              # getting the items purchased by these other users
                        if (user1 == user2): continue
                        meta_cosine_sim_score = round(self.get_cosine_similarity(user1_metavec, user2_metavec), 3)  # getting user-user cosine similarity
                        line_cosine_sim_score = round(self.get_cosine_similarity(user1_linevec, user2_linevec), 3)  # getting user-user cosine similarity
                        rec_items = [item for item in items2 if item not in items1]                                 # getting recommended items via each of these other users
                        self.meta_user_simscore[user2] = meta_cosine_sim_score
                        self.line_user_simscore[user2] = line_cosine_sim_score
                        self.user_rec[user2] = rec_items
                        weighted_cosine_sim_score = (meta_cosine_sim_score * default_weight_meta) + (line_cosine_sim_score * default_weight_line)                    
                        if user2 in friends1:
                            friends2 = set(self.user_friend.get(user2,[]))
                            mutual_friends = len(friends1 & friends2)
                            total_friends = len(friends1 | friends2)
                            friend_similarity = mutual_friends/total_friends
                            new_sim_score = weighted_cosine_sim_score + (friend_similarity * default_weight_friends)
                            self.user_simscore[user2] = new_sim_score
                        else:
                            new_weighted_cosine_sim_score = (meta_cosine_sim_score * 0.55) + (line_cosine_sim_score * 0.45)                    
                            self.user_simscore[user2] = new_weighted_cosine_sim_score
                l = sorted(self.user_simscore.items(), key=lambda x: x[1], reverse = True)
                top_five_users = [user for user,_ in l[0:5]]                                                        # getting the top 5 most similar users to the focal user
                top_five_users_items = set()                                                                        # now we fetch the recommended items for these top users
                for user in top_five_users:
                    items = self.user_rec[user]
                    for item in items:
                        top_five_users_items.add(item)
                self.first_user_recitems[user1] = list(top_five_users_items)
                l_meta = sorted(self.meta_user_simscore.items(), key=lambda x: x[1], reverse = True)
                top_five_meta_users = [user for user,_ in l_meta[0:5]]                                              # getting the top 5 most similar users to the focal user
                top_five_meta_users_items = set()                                                                   # now we fetch the recommended items for these top users
                for user in top_five_meta_users:
                    items = self.user_rec[user]
                    for item in items:
                        top_five_meta_users_items.add(item)
                l_line = sorted(self.line_user_simscore.items(), key=lambda x: x[1], reverse = True)
                top_five_line_users = [user for user,_ in l_line[0:5]]                                               # getting the top 5 most similar users to the focal user
                top_five_line_users_items = set()                                                                    # now we fetch the recommended items for these top users
                for user in top_five_line_users:
                    items = self.user_rec[user]
                    for item in items:
                        top_five_line_users_items.add(item)
                friends = self.user_friend.get(user1,[])
                rec_friend_items = []
                if len(friends) > 0:
                    for friend in friends:
                        friend_items = self.user_item.get(friend,[])
                        if len(friend_items) > 0:
                            rec_friend_items = [item for item in friend_items if item not in items1]
                final_rec_items = list(top_five_meta_users_items | top_five_line_users_items | set(rec_friend_items))
                self.second_user_recitems[user1] = final_rec_items    
        
    def get_accuracy_score(self):
        print("\n First Approach\n")
        sample_items = {k: self.first_user_recitems[k] for k in list(self.first_user_recitems)[:20]}
        print(sample_items)
        print("\n")
        count_2days = count_4days = count_10days = 0
        for user in self.first_user_recitems:
            recommended_items = set(self.first_user_recitems.get(user,[]))
            new_items_2days = set(self.test2days_user_item.get(user,[]))
            new_items_4days = set(self.test4days_user_item.get(user,[]))
            new_items_10days = set(self.test10days_user_item.get(user,[]))
            if len(recommended_items & new_items_2days) > 0:
                count_2days += 1
            if len(recommended_items & new_items_4days) > 0:
                count_4days += 1
            if len(recommended_items & new_items_10days) > 0:
                count_10days += 1
        print("Accuracy for Hybrid Recommender System")
        print("For a 2-day testing period:", count_2days/10000)
        print("For a 4-day testing period:", count_4days/10000)
        print("For a 10-day testing period:", count_10days/10000)
        print("\n Second Approach\n")
        sample_items = {k: self.second_user_recitems[k] for k in list(self.second_user_recitems)[:20]}
        print(sample_items)
        print("\n")
        count_2days = count_4days = count_10days = 0
        for user in self.second_user_recitems:
            recommended_items = set(self.second_user_recitems.get(user,[]))
            new_items_2days = set(self.test2days_user_item.get(user,[]))
            new_items_4days = set(self.test4days_user_item.get(user,[]))
            new_items_10days = set(self.test10days_user_item.get(user,[]))
            if len(recommended_items & new_items_2days) > 0:
                count_2days += 1
            if len(recommended_items & new_items_4days) > 0:
                count_4days += 1
            if len(recommended_items & new_items_10days) > 0:
                count_10days += 1
        print("Accuracy for Hybrid Recommender System")
        print("For a 2-day testing period:", count_2days/10000)
        print("For a 4-day testing period:", count_4days/10000)
        print("For a 10-day testing period:", count_10days/10000)
                            
                
dirpath = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Files'
outputfilename1 = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Latent Vector Representation/Gaming_output_metapath_UGIGU_w50_l20.txt'
outputfilename2 = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/LINE/LINE_vec_2nd_wo_norm.txt'


def main():
    hrs = HybridRecSys()
    hrs.read_data(dirpath)
    hrs.read_meta2vec_output(outputfilename1)
    hrs.read_LINE_output(outputfilename2)
    hrs.make_item_recommendations_selected_users()
    hrs.get_accuracy_score()

if __name__ == "__main__":
    main()

#users 1496096
#games 22
#items 3825
{'42465': ['33300'], '42535': ['33300'], '48889': ['33300'], '50613': ['33300', '33306'], '50643': ['33300', '33323'], '54583': ['33300'], '58043': ['33300'], '71509': ['33300', '33321'], '76127': ['33300'], '80377': ['33300'], '84133': ['33300'], '89961': ['33300', '33305', '33332'], '94437': ['33300', '33318'], '94577': ['33300', '33321', '33323'], '94629': ['33300', '33305'], '96461': ['33300'], '98003': ['33300'], '101481': ['33300', '33321', '33323'], '102363': ['33300', '33302', '33304'], '102763': ['33300']}


{'43639': ['89', '9001'], '44765': ['lolvn001', 'lolvn002'], '45121': ['MSTAR001', 'MSTAR004', 'MSTAR006'], '45693': ['tw_18_5'], '47495': ['loltw004'], '47503': ['145', '151', '2', '239', '43', '57', '9000', '9001', '9002'], '52609': ['lolvn003'], '54583': ['package_1', 'package_2', 'package_4'], '61621': ['tw_18_5'], '61647': ['RP001', 'RP002', 'RP003'], '64165': ['tw_18_1', 'tw_18_5'], '66201': ['15325'], '67707': ['lolvn002', 'lolvn


 First Approach

{'11493857': ['loltw001', 'loltw003', 'loltw004', 'loltw005'], '10842607': ['loltw001', 'loltw003', 'loltw002'], '6540767': ['package_9', 'CISLUCK11', 'CISLUCK1'], '11722909': ['tw_18_3', 'tw_18_5', 'tw_18_2'], '33909557': ['15373', 'loltw001', 'loltw003'], '135957': ['loltw001', 'loltw003', 'loltw004'], '11047793': ['loltw003', '15238', '15386', 'loltw001', 'loltw004'], '43444903': ['lolvn002', 'lolvn003'], '16912887': ['RP002', 'RP001'], '37896045': ['lolvn002', '13603', '12959', '13469', 'lolvn001', 'lolvn003'], '14309347': ['lolth007', 'lolth006', 'lolth004', 'lolth005'], '23027837': ['loltw001', '15029', 'loltw003', '15137'], '6387099': ['loltw001', '15262', 'loltw003', 'loltw004'], '7990961': ['RP003', 'elph001', 'RP002', 'RP001'], '31866147': ['loltw001', '15266', 'loltw004', '15351'], '33673571': ['RP003', 'RP004', 'RP002', 'RP001'], '38494617': ['lolvn002', '12983', '14194', 'lolvn001'], '42186575': ['fo3vn002', 'fo3vn005', 'fo3vn004', 'fo3vn003'], '39908261'

In [1]:
import sys
import os
import random
import numpy as np
from collections import Counter

class HybridRecSys:
    def __init__(self):
        self.id_user = dict()
        self.user_id = dict()
        self.id_game = dict()
        self.game_id = dict()
        self.id_item = dict()
        self.item_id = dict()
        self.item_user = dict()
        self.user_item = dict()
        self.game_user = dict()
        self.user_game = dict()
        self.game_item = dict()
        self.item_game = dict()
        self.test2days_item_user = dict()
        self.test2days_user_item = dict()
        self.test4days_item_user = dict()
        self.test4days_user_item = dict()
        self.test10days_item_user = dict()
        self.test10days_user_item = dict()
        self.user_friend = dict()
        self.userlist = []
        self.selected_user_items = dict()

    def read_data(self, dirpath):
        with open(dirpath + "/id_user.txt") as udictfile:
            for line in udictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_user[toks[0]] = toks[1].replace(" ", "")    # {uid1: User1, uid2: User2....}
                    self.user_id[toks[1]] = toks[0].replace(" ", "")    # {User1: uid1, User2: uid2....}

        print("#users", len(self.id_user))

        with open(dirpath + "/id_game.txt") as gdictfile:
            for line in gdictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_game[toks[0]] = toks[1].replace(" ", "")    # {33302: LoLsg, 33305: LoLmy....}
                    self.game_id[toks[1]] = toks[0].replace(" ", "")    # {LoLsg: 33302, LoLmy: 33305....}

        print("#games", len(self.id_game))
        
        with open(dirpath + "/id_item.txt") as idictfile:
            for line in idictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_item[toks[0]] = toks[1].replace(" ", "")    # {itemid1: Item1, itemid2: Item2....}
                    self.item_id[toks[1]] = toks[0].replace(" ", "")    # {Item1: itemid1, Item2: itemid2....}

        print("#items", len(self.id_item)) # comprehensive list of items

        with open(dirpath + "/game_user.txt") as gufile:  # data pulled from user game activity for 20 days
            for line in gufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    g, u = toks[0], toks[1]
                    if g not in self.game_user:
                        self.game_user[g] = []
                    self.game_user[g].append(u)   # {gameid1: [uid1,uid2,...], gameid2: [uid2,uid3,....]} = dict of list of users playing games
                    if u not in self.user_game:
                        self.user_game[u] = []
                    self.user_game[u].append(g)   # {uid1: [gameid1,gameid2,...], uid2: [gameid2,gameid3,....]} = dict of list of games played by each user
        
        
        with open(dirpath + "/item_user.txt") as iufile:  # list of all items purchased by users
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.item_user:
                        self.item_user[i] = []
                    self.item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.user_item:
                        self.user_item[u] = []
                    self.user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        
        
        with open(dirpath + "/item_game.txt") as igfile:  # list of all in-game items
            for line in igfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, g = toks[0], toks[1]
                    if i not in self.item_game:
                        self.item_game[i] = []  
                    self.item_game[i].append(g) # {itemid1: [gameid1,gameid2,...], itemid2: [gameid2,gameid3,....]} = list of items that got recommended. Every item can be in multiple games
                    if g not in self.game_item:
                        self.game_item[g] = []
                    self.game_item[g].append(i) # {gameid1: [itemid1,itemid2,...], gameid2: [itemd2,itemd3,....]} = list of items available in-game for purchase
        
        with open(dirpath + "/test2days_item_user.txt") as iufile:  # list of all items purchased by users in 2 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test2days_item_user:
                        self.test2days_item_user[i] = []
                    self.test2days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test2days_user_item:
                        self.test2days_user_item[u] = []
                    self.test2days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 2 days of test period:", len(self.test2days_user_item))
        
        with open(dirpath + "/test4days_item_user.txt") as iufile:  # list of all items purchased by users in 4 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test4days_item_user:
                        self.test4days_item_user[i] = []
                    self.test4days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test4days_user_item:
                        self.test4days_user_item[u] = []
                    self.test4days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 4 days of test period:", len(self.test4days_user_item))
        
        with open(dirpath + "/test10days_item_user.txt") as iufile:  # list of all items purchased by users in 10 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test10days_item_user:
                        self.test10days_item_user[i] = []
                    self.test10days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test10days_user_item:
                        self.test10days_user_item[u] = []
                    self.test10days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 10 days of test period:", len(self.test10days_user_item))
        
        with open(dirpath + "/selected_user.txt") as sufile:  # pull the randomly selected users
            for user in sufile:
                self.userlist.append(user.strip())
                
        with open(dirpath + "/user_friend.txt") as uffile:  # list of all friendship linkages
            for line in uffile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    u, f = toks[0], toks[1]
                    if u not in self.user_friend:
                        self.user_friend[u] = []
                    self.user_friend[u].append(f)   # {uid1: [friendid1,friendid2,...], uid2: [friendid2,friendid3,....]} = dict of each person's friends
        
    def read_meta2vec_output(self, outputfilename):
        self.user_vec = dict()
        self.game_vec = dict()
        self.item_vec = dict()
        with open(outputfilename) as nvfile:             
            for line in nvfile:
                toks = line.strip().split(" ")
                if len(toks) == 129:
                    node = toks[0]
                    del toks[0]
                    if node[0] == 'u': 
                        self.user_vec[self.user_id[node]] = toks    # {uid1: [user_vector_values], uid2: [user_vector_values]..}
                    elif node[0] == 'g':
                        self.game_vec[self.game_id[node]] = toks    # {gameid1: [game_vector_values], gameid2: [game_vector_values]..}
                    elif node[0] == 'i':
                        self.item_vec[self.item_id[node]] = toks    # {itemid1: [item_vector_values], itemid2: [item_vector_values]..}
        
    def read_LINE_output(self, outputfilename):
        self.user_line_vec = dict()
        with open(outputfilename) as nvfile:             
            for line in nvfile:
                toks = line.strip().split(" ")
                if len(toks) == 129:
                    node = toks[0]
                    del toks[0] 
                    self.user_line_vec[node] = toks    # {uid1: [user_vector_values], uid2: [user_vector_values]..}
        
    def get_cosine_similarity(self, vec1, vec2):
        a = np.array(vec1, dtype=float)
        b = np.array(vec2, dtype=float)
        dot_product = np.dot(a, b)
        norm_a = np.linalg.norm(a)
        norm_b = np.linalg.norm(b)
        return dot_product / (norm_a * norm_b)
    
    def make_item_recommendations_selected_users(self):
        top_5_user_recitems = dict()
        top_10_user_recitems = dict()
        top_15_user_recitems = dict()
        default_weight_meta = 0.43
        default_weight_line = 0.38
        default_weight_friends = 0.19
        print("\nMaking Recommendations\n")
        print("#Number of users randomly selected:", len(self.userlist))
        for user1 in self.userlist:                                                                                # iterating through 10,000 randomly selected users
            user1_metavec = self.user_vec.get(user1,[])                                                            # getting the latent vector representation
            user1_linevec = self.user_line_vec.get(user1,[])
            if (user1_metavec != [] and user1_linevec !=[]):
                self.user_simscore = {}
                self.user_rec = {}
                items1 = self.user_item.get(user1,[])                                                               # getting the items purchased
                friends1 = set(self.user_friend.get(user1,[]))
                for user2 in self.user_item:                                                                        # iterating through other users in the gaming network
                    user2_metavec = self.user_vec.get(user2,[])                                                     # getting the latent vector representation
                    user2_linevec = self.user_line_vec.get(user2,[])
                    if (user2_metavec != [] and user2_linevec !=[]):
                        items2 = self.user_item[user2]                                                              # getting the items purchased by these other users
                        if (user1 == user2): continue
                        meta_cosine_sim_score = round(self.get_cosine_similarity(user1_metavec, user2_metavec), 3)  # getting user-user cosine similarity
                        line_cosine_sim_score = round(self.get_cosine_similarity(user1_linevec, user2_linevec), 3)  # getting user-user cosine similarity
                        rec_items = [item for item in items2 if item not in items1]                                 # getting recommended items via each of these other users
                        self.user_rec[user2] = rec_items
                        weighted_cosine_sim_score = (meta_cosine_sim_score * default_weight_meta) + (line_cosine_sim_score * default_weight_line)                    
                        if user2 in friends1:
                            friends2 = set(self.user_friend.get(user2,[]))
                            mutual_friends = len(friends1 & friends2)
                            total_friends = len(friends1 | friends2)
                            friend_similarity = mutual_friends/total_friends
                            new_sim_score = weighted_cosine_sim_score + (friend_similarity * default_weight_friends)
                            self.user_simscore[user2] = new_sim_score
                        else:
                            new_weighted_cosine_sim_score = (meta_cosine_sim_score * 0.55) + (line_cosine_sim_score * 0.45)                    
                            self.user_simscore[user2] = new_weighted_cosine_sim_score
                self.sorted_user_simscore = sorted(self.user_simscore.items(), key=lambda x: x[1], reverse = True)
                top_5_user_recitems[user1] = self.get_top_k_user_items(k=5)
                top_10_user_recitems[user1] = self.get_top_k_user_items(k=10)
                top_15_user_recitems[user1] = self.get_top_k_user_items(k=15)
        sample_items = {k: top_5_user_recitems[k] for k in list(top_5_user_recitems)[:20]}
        print("\n")
        print("Accuracy for Hybrid Recommender System based on Top-5 similar users")
        self.get_accuracy_score(user_recitems = top_5_user_recitems)
        sample_items = {k: top_10_user_recitems[k] for k in list(top_10_user_recitems)[:20]}
        print("\n")
        print("Accuracy for Hybrid Recommender System based on Top-10 similar users")
        self.get_accuracy_score(user_recitems = top_10_user_recitems)
        sample_items = {k: top_15_user_recitems[k] for k in list(top_15_user_recitems)[:20]}
        print("\n")
        print("Accuracy for Hybrid Recommender System based on Top-15 similar users")
        self.get_accuracy_score(user_recitems = top_15_user_recitems)    
        
    def get_top_k_user_items(self, k):
        top_k_users = [user for user,_ in self.sorted_user_simscore[0:k]]   # getting the top k most similar users to the focal user
        top_k_users_items = set()                                           # now we fetch the recommended items for these top users
        for user in top_k_users:
            items = self.user_rec[user]
            for item in items:
                top_k_users_items.add(item)
        return list(top_k_users_items)      
        
    def get_accuracy_score(self, user_recitems):
        count_2days = count_4days = count_10days = 0
        for user in user_recitems:
            recommended_items = set(user_recitems.get(user,[]))
            new_items_2days = set(self.test2days_user_item.get(user,[]))
            new_items_4days = set(self.test4days_user_item.get(user,[]))
            new_items_10days = set(self.test10days_user_item.get(user,[]))
            if len(recommended_items & new_items_2days) > 0:
                count_2days += 1
            if len(recommended_items & new_items_4days) > 0:
                count_4days += 1
            if len(recommended_items & new_items_10days) > 0:
                count_10days += 1
        print("For a 2-day testing period:", count_2days/10000)
        print("For a 4-day testing period:", count_4days/10000)
        print("For a 10-day testing period:", count_10days/10000)
                            
                
dirpath = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Files'
outputfilename1 = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Latent Vector Representation/Gaming_output_metapath_UGIGU_w50_l20.txt'
outputfilename2 = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/LINE/LINE_vec_2nd_wo_norm.txt'


def main():
    hrs = HybridRecSys()
    hrs.read_data(dirpath)
    hrs.read_meta2vec_output(outputfilename1)
    hrs.read_LINE_output(outputfilename2)
    hrs.make_item_recommendations_selected_users()

if __name__ == "__main__":
    main()

#users 1496096
#games 22
#items 3825
No of users who bought items in first 2 days of test period: 2700
No of users who bought items in first 4 days of test period: 4999
No of users who bought items in first 10 days of test period: 10302

Making Recommendations

#Number of users randomly selected: 10000


Accuracy for Hybrid Recommender System based on Top-5 similar users
For a 2-day testing period: 0.0066
For a 4-day testing period: 0.014
For a 10-day testing period: 0.0291


Accuracy for Hybrid Recommender System based on Top-10 similar users
For a 2-day testing period: 0.0082
For a 4-day testing period: 0.0167
For a 10-day testing period: 0.0352


Accuracy for Hybrid Recommender System based on Top-15 similar users
For a 2-day testing period: 0.0086
For a 4-day testing period: 0.0176
For a 10-day testing period: 0.0371


In [1]:
import sys
import os
import random
import numpy as np
from collections import Counter

class HybridRecSys:
    def __init__(self):
        self.id_user = dict()
        self.user_id = dict()
        self.id_game = dict()
        self.game_id = dict()
        self.id_item = dict()
        self.item_id = dict()
        self.item_user = dict()
        self.user_item = dict()
        self.game_user = dict()
        self.user_game = dict()
        self.game_item = dict()
        self.item_game = dict()
        self.test2days_item_user = dict()
        self.test2days_user_item = dict()
        self.test4days_item_user = dict()
        self.test4days_user_item = dict()
        self.test10days_item_user = dict()
        self.test10days_user_item = dict()
        self.user_friend = dict()
        self.userlist = []
        self.star_items = []
        self.mid_items = []
        self.tail_items = []

    def read_data(self, dirpath):
        with open(dirpath + "/id_user.txt") as udictfile:
            for line in udictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_user[toks[0]] = toks[1].replace(" ", "")    # {uid1: User1, uid2: User2....}
                    self.user_id[toks[1]] = toks[0].replace(" ", "")    # {User1: uid1, User2: uid2....}

        print("#users", len(self.id_user))

        with open(dirpath + "/id_game.txt") as gdictfile:
            for line in gdictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_game[toks[0]] = toks[1].replace(" ", "")    # {33302: LoLsg, 33305: LoLmy....}
                    self.game_id[toks[1]] = toks[0].replace(" ", "")    # {LoLsg: 33302, LoLmy: 33305....}

        print("#games", len(self.id_game))
        
        with open(dirpath + "/id_item.txt") as idictfile:
            for line in idictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_item[toks[0]] = toks[1].replace(" ", "")    # {itemid1: Item1, itemid2: Item2....}
                    self.item_id[toks[1]] = toks[0].replace(" ", "")    # {Item1: itemid1, Item2: itemid2....}

        print("#items", len(self.id_item)) # comprehensive list of items

        with open(dirpath + "/game_user.txt") as gufile:  # data pulled from user game activity for 20 days
            for line in gufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    g, u = toks[0], toks[1]
                    if g not in self.game_user:
                        self.game_user[g] = []
                    self.game_user[g].append(u)   # {gameid1: [uid1,uid2,...], gameid2: [uid2,uid3,....]} = dict of list of users playing games
                    if u not in self.user_game:
                        self.user_game[u] = []
                    self.user_game[u].append(g)   # {uid1: [gameid1,gameid2,...], uid2: [gameid2,gameid3,....]} = dict of list of games played by each user
        
        
        with open(dirpath + "/item_user.txt") as iufile:  # list of all items purchased by users
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.item_user:
                        self.item_user[i] = []
                    self.item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.user_item:
                        self.user_item[u] = []
                    self.user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        
        
        with open(dirpath + "/item_game.txt") as igfile:  # list of all in-game items
            for line in igfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, g = toks[0], toks[1]
                    if i not in self.item_game:
                        self.item_game[i] = []  
                    self.item_game[i].append(g) # {itemid1: [gameid1,gameid2,...], itemid2: [gameid2,gameid3,....]} = list of items that got recommended. Every item can be in multiple games
                    if g not in self.game_item:
                        self.game_item[g] = []
                    self.game_item[g].append(i) # {gameid1: [itemid1,itemid2,...], gameid2: [itemd2,itemd3,....]} = list of items available in-game for purchase
        
        with open(dirpath + "/test2days_item_user.txt") as iufile:  # list of all items purchased by users in 2 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test2days_item_user:
                        self.test2days_item_user[i] = []
                    self.test2days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test2days_user_item:
                        self.test2days_user_item[u] = []
                    self.test2days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 2 days of test period:", len(self.test2days_user_item))
        
        with open(dirpath + "/test4days_item_user.txt") as iufile:  # list of all items purchased by users in 4 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test4days_item_user:
                        self.test4days_item_user[i] = []
                    self.test4days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test4days_user_item:
                        self.test4days_user_item[u] = []
                    self.test4days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 4 days of test period:", len(self.test4days_user_item))
        
        with open(dirpath + "/test10days_item_user.txt") as iufile:  # list of all items purchased by users in 10 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test10days_item_user:
                        self.test10days_item_user[i] = []
                    self.test10days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test10days_user_item:
                        self.test10days_user_item[u] = []
                    self.test10days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 10 days of test period:", len(self.test10days_user_item))
        
        with open(dirpath + "/selected_user.txt") as sufile:  # pull the randomly selected users
            for user in sufile:
                self.userlist.append(user.strip())
                
        with open(dirpath + "/user_friend.txt") as uffile:  # list of all friendship linkages
            for line in uffile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    u, f = toks[0], toks[1]
                    if u not in self.user_friend:
                        self.user_friend[u] = []
                    self.user_friend[u].append(f)   # {uid1: [friendid1,friendid2,...], uid2: [friendid2,friendid3,....]} = dict of each person's friends
        
    def read_meta2vec_output(self, outputfilename):
        self.user_vec = dict()
        self.game_vec = dict()
        self.item_vec = dict()
        with open(outputfilename) as nvfile:             
            for line in nvfile:
                toks = line.strip().split(" ")
                if len(toks) == 129:
                    node = toks[0]
                    del toks[0]
                    if node[0] == 'u': 
                        self.user_vec[self.user_id[node]] = toks    # {uid1: [user_vector_values], uid2: [user_vector_values]..}
                    elif node[0] == 'g':
                        self.game_vec[self.game_id[node]] = toks    # {gameid1: [game_vector_values], gameid2: [game_vector_values]..}
                    elif node[0] == 'i':
                        self.item_vec[self.item_id[node]] = toks    # {itemid1: [item_vector_values], itemid2: [item_vector_values]..}
        
    def read_LINE_output(self, outputfilename):
        self.user_line_vec = dict()
        with open(outputfilename) as nvfile:             
            for line in nvfile:
                toks = line.strip().split(" ")
                if len(toks) == 129:
                    node = toks[0]
                    del toks[0] 
                    self.user_line_vec[node] = toks    # {uid1: [user_vector_values], uid2: [user_vector_values]..}
        
    def get_cosine_similarity(self, vec1, vec2):
        a = np.array(vec1, dtype=float)
        b = np.array(vec2, dtype=float)
        dot_product = np.dot(a, b)
        norm_a = np.linalg.norm(a)
        norm_b = np.linalg.norm(b)
        return dot_product / (norm_a * norm_b)
    
    def make_user_recommendations_selected_cluster(self, cluster_items, flag):
        top_5_item_recusers = dict()
        top_10_item_recusers = dict()
        top_15_item_recusers = dict()
        top_20_item_recusers = dict()
        if (flag == 'S'):
            default_weight_meta = 0.39
            default_weight_line = 0.22
            default_weight_friends = 0.39
        if (flag == 'T'):
            default_weight_meta = 0.17
            default_weight_line = 0.22
            default_weight_friends = 0.61
        for item in cluster_items:
            user_simscore = {}
            temp_user_simscore = dict()
            users = self.item_user.get(item,[])
            if (users != []):
                for user1 in users:
                    user1_metavec = self.user_vec.get(user1,[])                                                            
                    user1_linevec = self.user_line_vec.get(user1,[])
                    friends1 = set(self.user_friend.get(user1,[]))
                    items1 = self.user_item.get(user1,[])
                    for user2 in self.userlist:
                        user2_metavec = self.user_vec.get(user2,[])                                                     
                        user2_linevec = self.user_line_vec.get(user2,[])
                        items2 = self.user_item.get(user2,[])
                        if (user1 == user2): continue
                        if item not in items2:
                            friends2 = set(self.user_friend.get(user2,[]))
                            mutual_friends = len(friends1 & friends2)
                            total_friends = len(friends1 | friends2)
                            if (total_friends > 0):
                                friend_similarity_score = mutual_friends/total_friends
                            else:
                                friend_similarity_score = 0
                            if (user1_metavec != []) and (user2_metavec != []):
                                meta_cosine_sim_score = round(self.get_cosine_similarity(user1_metavec, user2_metavec), 3)
                            else:
                                meta_cosine_sim_score = 0
                            if (user1_linevec != []) and (user2_linevec != []):
                                line_cosine_sim_score = round(self.get_cosine_similarity(user1_linevec, user2_linevec), 3)
                            else:
                                line_cosine_sim_score = 0
                            weighted_sim_score = (meta_cosine_sim_score * default_weight_meta) + (line_cosine_sim_score * default_weight_line) + (friend_similarity_score * default_weight_friends)
                            if user2 not in temp_user_simscore:
                                temp_user_simscore[user2] = []
                            temp_user_simscore[user2].append(weighted_sim_score)
                for user in temp_user_simscore:
                    weighted_sim_values = temp_user_simscore[user]
                    user_simscore[user] = max(weighted_sim_values)
            sorted_user_simscore = sorted(user_simscore.items(), key=lambda x: x[1], reverse = True)
            top_5_item_recusers[item] = [user for user,_ in sorted_user_simscore[0:5]]
            top_10_item_recusers[item] = [user for user,_ in sorted_user_simscore[0:10]]
            top_15_item_recusers[item] = [user for user,_ in sorted_user_simscore[0:15]]
            top_20_item_recusers[item] = [user for user,_ in sorted_user_simscore[0:20]]
        sample_users = {k: top_5_item_recusers[k] for k in list(top_5_item_recusers)[:20]}
        #print(sample_users)
        #print("\n")
        sample_users = {k: top_10_item_recusers[k] for k in list(top_10_item_recusers)[:20]}
        #print(sample_users)
        #print("\n")
        sample_users = {k: top_15_item_recusers[k] for k in list(top_15_item_recusers)[:20]}
        #print(sample_users)
        #print("\n")
        print("Accuracy for Hybrid Recommender System based on Top-5 similar users")
        self.get_accuracy_score(item_recusers = top_5_item_recusers, cluster_items = cluster_items)
        print("Accuracy for Hybrid Recommender System based on Top-10 similar users")
        self.get_accuracy_score(item_recusers = top_10_item_recusers, cluster_items = cluster_items)
        print("Accuracy for Hybrid Recommender System based on Top-15 similar users")
        self.get_accuracy_score(item_recusers = top_15_item_recusers, cluster_items = cluster_items)
        print("Accuracy for Hybrid Recommender System based on Top-20 similar users")
        self.get_accuracy_score(item_recusers = top_20_item_recusers, cluster_items = cluster_items)
    
        
    def get_accuracy_score(self, item_recusers, cluster_items):
        count_2days = count_4days = count_10days = 0
        for item in cluster_items:
            counter_2days = counter_4days = counter_10days = 0
            recommended_users = item_recusers.get(item,[])
            for user in recommended_users:
                new_items_2days = self.test2days_user_item.get(user,[])
                new_items_4days = self.test4days_user_item.get(user,[])
                new_items_10days = self.test10days_user_item.get(user,[])
                if item in new_items_2days:
                    counter_2days += 1
                if item in new_items_4days:
                    counter_4days += 1
                if item in new_items_10days:
                    counter_10days += 1        
            if (counter_2days > 0):
                count_2days += 1
            if (counter_4days > 0):
                count_4days += 1
            if (counter_10days > 0):
                count_10days += 1
        print("For a 2-day testing period:", count_2days/len(cluster_items))
        print("For a 4-day testing period:", count_4days/len(cluster_items))
        print("For a 10-day testing period:", count_10days/len(cluster_items))
        
    
    def make_item_recommendations(self):
        with open(dirpath + "/rank_item_quantity_price_sales.txt") as itemfile:  # pull the ranked items, and create clusters
            for line in itemfile:
                toks = line.strip().split("\t")
                if len(toks) == 5:
                    rank = int(toks[0])
                    if rank <= 40:
                        self.star_items.append(toks[1])
                    else:
                        self.tail_items.append(toks[1])
        print(self.star_items)
        print(self.tail_items)
        print("\nMaking Recommendations\n")
        print("\nAccuracy for Star Items\n")
        self.make_user_recommendations_selected_cluster(cluster_items = self.star_items, flag = 'S')
        print("\nAccuracy for Long Tail Items\n")
        self.make_user_recommendations_selected_cluster(cluster_items = self.tail_items, flag = 'T')                           
                
dirpath = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Files'
outputfilename1 = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Latent Vector Representation/Gaming_output_metapath_UGIGU_w50_l20.txt'
outputfilename2 = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/LINE/LINE_vec_2nd_wo_norm.txt'


def main():
    hrs = HybridRecSys()
    hrs.read_data(dirpath)
    hrs.read_meta2vec_output(outputfilename1)
    hrs.read_LINE_output(outputfilename2)
    hrs.make_item_recommendations()

if __name__ == "__main__":
    main()

#users 1496096
#games 22
#items 3825
No of users who bought items in first 2 days of test period: 2700
No of users who bought items in first 4 days of test period: 4999
No of users who bought items in first 10 days of test period: 10302
['lolvn002', 'lolvn001', 'lolvn003', 'loltw001', 'RP003', 'loltw003', 'RP002', 'loltw004', 'lolvn004', 'RP001', 'package_20', 'loltw005', 'tw_18_3', '89', 'loltw002', 'tw_18_2', 'tw_18_5', 'lolsam001', '9002', 'lolsam003', 'fo3vn003', 'RP004', 'FO3TH003', 'fo3vn004', 'tw_18_4', 'package_4', '9001', 'package_2', 'fo3vn002', 'lolsam004', 'tw_19_3', 'package_18', 'package_19', 'package_1', 'tw_19_2', '166', '145', 'fo3vn005', '9000', '33']
['43', '31', 'RP005', '237', 'HONLUCK1', '57', '2', 'lolsam006', 'lolsam002', '239', 'FO3TH004', 'FO3TH006', 'tw_19_5', 'lolvn005', '157', 'pbth001', '159', '205', 'lolth004', '228', '280', 'tw_19_4', 'lolth001', 'lolsam005', '138', 'lolth003', 'tw_18_1', 'tw_15_5', '34', 'tw_15_4', '188', '25', '192', '200', 'FO3TH005',

In [2]:
# Long Tail Analysis - Optimal Hybrid RecSys
import sys
import os
import random
import numpy as np
from collections import Counter

class HybridRecSys:
    def __init__(self):
        self.id_user = dict()
        self.user_id = dict()
        self.id_game = dict()
        self.game_id = dict()
        self.id_item = dict()
        self.item_id = dict()
        self.item_user = dict()
        self.user_item = dict()
        self.game_user = dict()
        self.user_game = dict()
        self.game_item = dict()
        self.item_game = dict()
        self.test2days_item_user = dict()
        self.test2days_user_item = dict()
        self.test4days_item_user = dict()
        self.test4days_user_item = dict()
        self.test10days_item_user = dict()
        self.test10days_user_item = dict()
        self.user_friend = dict()
        self.userlist = []
        self.selected_user_items = dict()
        self.star_items = []
        self.tail_items = []
        self.all_items = []

    def read_data(self, dirpath):
        with open(dirpath + "/id_user.txt") as udictfile:
            for line in udictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_user[toks[0]] = toks[1].replace(" ", "")    # {uid1: User1, uid2: User2....}
                    self.user_id[toks[1]] = toks[0].replace(" ", "")    # {User1: uid1, User2: uid2....}

        print("#users", len(self.id_user))

        with open(dirpath + "/id_game.txt") as gdictfile:
            for line in gdictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_game[toks[0]] = toks[1].replace(" ", "")    # {33302: LoLsg, 33305: LoLmy....}
                    self.game_id[toks[1]] = toks[0].replace(" ", "")    # {LoLsg: 33302, LoLmy: 33305....}

        print("#games", len(self.id_game))
        
        with open(dirpath + "/id_item.txt") as idictfile:
            for line in idictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_item[toks[0]] = toks[1].replace(" ", "")    # {itemid1: Item1, itemid2: Item2....}
                    self.item_id[toks[1]] = toks[0].replace(" ", "")    # {Item1: itemid1, Item2: itemid2....}

        print("#items", len(self.id_item)) # comprehensive list of items

        with open(dirpath + "/game_user.txt") as gufile:  # data pulled from user game activity for 20 days
            for line in gufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    g, u = toks[0], toks[1]
                    if g not in self.game_user:
                        self.game_user[g] = []
                    self.game_user[g].append(u)   # {gameid1: [uid1,uid2,...], gameid2: [uid2,uid3,....]} = dict of list of users playing games
                    if u not in self.user_game:
                        self.user_game[u] = []
                    self.user_game[u].append(g)   # {uid1: [gameid1,gameid2,...], uid2: [gameid2,gameid3,....]} = dict of list of games played by each user
        
        
        with open(dirpath + "/item_user.txt") as iufile:  # list of all items purchased by users
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.item_user:
                        self.item_user[i] = []
                    self.item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.user_item:
                        self.user_item[u] = []
                    self.user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        
        
        with open(dirpath + "/item_game.txt") as igfile:  # list of all in-game items
            for line in igfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, g = toks[0], toks[1]
                    if i not in self.item_game:
                        self.item_game[i] = []  
                    self.item_game[i].append(g) # {itemid1: [gameid1,gameid2,...], itemid2: [gameid2,gameid3,....]} = list of items that got recommended. Every item can be in multiple games
                    if g not in self.game_item:
                        self.game_item[g] = []
                    self.game_item[g].append(i) # {gameid1: [itemid1,itemid2,...], gameid2: [itemd2,itemd3,....]} = list of items available in-game for purchase
        
        with open(dirpath + "/test2days_item_user.txt") as iufile:  # list of all items purchased by users in 2 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test2days_item_user:
                        self.test2days_item_user[i] = []
                    self.test2days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test2days_user_item:
                        self.test2days_user_item[u] = []
                    self.test2days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 2 days of test period:", len(self.test2days_user_item))
        
        with open(dirpath + "/test4days_item_user.txt") as iufile:  # list of all items purchased by users in 4 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test4days_item_user:
                        self.test4days_item_user[i] = []
                    self.test4days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test4days_user_item:
                        self.test4days_user_item[u] = []
                    self.test4days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 4 days of test period:", len(self.test4days_user_item))
        
        with open(dirpath + "/test10days_item_user.txt") as iufile:  # list of all items purchased by users in 10 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test10days_item_user:
                        self.test10days_item_user[i] = []
                    self.test10days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test10days_user_item:
                        self.test10days_user_item[u] = []
                    self.test10days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 10 days of test period:", len(self.test10days_user_item))
        
        with open(dirpath + "/selected_user.txt") as sufile:  # pull the randomly selected users
            for user in sufile:
                self.userlist.append(user.strip())
                
        with open(dirpath + "/user_friend.txt") as uffile:  # list of all friendship linkages
            for line in uffile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    u, f = toks[0], toks[1]
                    if u not in self.user_friend:
                        self.user_friend[u] = []
                    self.user_friend[u].append(f)   # {uid1: [friendid1,friendid2,...], uid2: [friendid2,friendid3,....]} = dict of each person's friends
        
    def read_meta2vec_output(self, outputfilename):
        self.user_vec = dict()
        self.game_vec = dict()
        self.item_vec = dict()
        with open(outputfilename) as nvfile:             
            for line in nvfile:
                toks = line.strip().split(" ")
                if len(toks) == 129:
                    node = toks[0]
                    del toks[0]
                    if node[0] == 'u': 
                        self.user_vec[self.user_id[node]] = toks    # {uid1: [user_vector_values], uid2: [user_vector_values]..}
                    elif node[0] == 'g':
                        self.game_vec[self.game_id[node]] = toks    # {gameid1: [game_vector_values], gameid2: [game_vector_values]..}
                    elif node[0] == 'i':
                        self.item_vec[self.item_id[node]] = toks    # {itemid1: [item_vector_values], itemid2: [item_vector_values]..}
        
    def read_LINE_output(self, outputfilename):
        self.user_line_vec = dict()
        with open(outputfilename) as nvfile:             
            for line in nvfile:
                toks = line.strip().split(" ")
                if len(toks) == 129:
                    node = toks[0]
                    del toks[0] 
                    self.user_line_vec[node] = toks    # {uid1: [user_vector_values], uid2: [user_vector_values]..}
        
    def get_cosine_similarity(self, vec1, vec2):
        a = np.array(vec1, dtype=float)
        b = np.array(vec2, dtype=float)
        dot_product = np.dot(a, b)
        norm_a = np.linalg.norm(a)
        norm_b = np.linalg.norm(b)
        return dot_product / (norm_a * norm_b)
    
    def make_user_recommendations_selected_cluster(self, cluster_items, flag):
        top_5_user_recitems = dict()
        top_10_user_recitems = dict()
        top_15_user_recitems = dict()
        top_20_user_recitems = dict()
        if (flag == 'S'):
            default_weight_meta = 0.44
            default_weight_line = 0.40
            default_weight_friends = 0.16
        if (flag == 'T'):
            default_weight_meta = 0.46
            default_weight_line = 0.38
            default_weight_friends = 0.16
        print("\nMaking Recommendations\n")
        print("#Number of users randomly selected:", len(self.userlist))
        for user1 in self.userlist:                                                                                # iterating through 10,000 randomly selected users
            user1_metavec = self.user_vec.get(user1,[])                                                            # getting the latent vector representation
            user1_linevec = self.user_line_vec.get(user1,[])
            if (user1_metavec != [] and user1_linevec !=[]):
                self.user_simscore = {}
                self.user_rec = {}
                items1 = self.user_item.get(user1,[])                                                               # getting the items purchased
                friends1 = set(self.user_friend.get(user1,[]))
                for user2 in self.user_item:                                                                        # iterating through other users in the gaming network
                    user2_metavec = self.user_vec.get(user2,[])                                                     # getting the latent vector representation
                    user2_linevec = self.user_line_vec.get(user2,[])
                    if (user2_metavec != [] and user2_linevec !=[]):
                        items2 = self.user_item[user2]                                                              # getting the items purchased by these other users
                        if (user1 == user2): continue
                        rec_items = []
                        meta_cosine_sim_score = round(self.get_cosine_similarity(user1_metavec, user2_metavec), 3)  # getting user-user cosine similarity
                        line_cosine_sim_score = round(self.get_cosine_similarity(user1_linevec, user2_linevec), 3)  # getting user-user cosine similarity
                        for item in cluster_items:
                            if item not in items1:
                                if item in items2:
                                    rec_items.append(item)
                        self.user_rec[user2] = rec_items
                        weighted_cosine_sim_score = (meta_cosine_sim_score * default_weight_meta) + (line_cosine_sim_score * default_weight_line)                    
                        if user2 in friends1:
                            friends2 = set(self.user_friend.get(user2,[]))
                            mutual_friends = len(friends1 & friends2)
                            total_friends = len(friends1 | friends2)
                            friend_similarity = mutual_friends/total_friends
                            new_sim_score = weighted_cosine_sim_score + (friend_similarity * default_weight_friends)
                            self.user_simscore[user2] = new_sim_score
                        else:
                            new_weighted_cosine_sim_score = (meta_cosine_sim_score * 0.52) + (line_cosine_sim_score * 0.48)                    
                            self.user_simscore[user2] = new_weighted_cosine_sim_score
                self.sorted_user_simscore = sorted(self.user_simscore.items(), key=lambda x: x[1], reverse = True)
                top_5_user_recitems[user1] = self.get_top_k_user_items(k=5)
                top_10_user_recitems[user1] = self.get_top_k_user_items(k=10)
                top_15_user_recitems[user1] = self.get_top_k_user_items(k=15)
                top_20_user_recitems[user1] = self.get_top_k_user_items(k=20)
        print("\n")
        print("Accuracy for Hybrid Recommender System based on Top-5 similar users")
        self.get_accuracy_score(user_recitems = top_5_user_recitems)
        print("\n")
        print("Accuracy for Hybrid Recommender System based on Top-10 similar users")
        self.get_accuracy_score(user_recitems = top_10_user_recitems)
        print("\n")
        print("Accuracy for Hybrid Recommender System based on Top-15 similar users")
        self.get_accuracy_score(user_recitems = top_15_user_recitems)  
        print("\n")
        print("Accuracy for Hybrid Recommender System based on Top-20 similar users")
        self.get_accuracy_score(user_recitems = top_20_user_recitems)
        
    def get_top_k_user_items(self, k):
        top_k_users = [user for user,_ in self.sorted_user_simscore[0:k]]   # getting the top k most similar users to the focal user
        top_k_users_items = set()                                           # now we fetch the recommended items for these top users
        for user in top_k_users:
            items = self.user_rec[user]
            for item in items:
                top_k_users_items.add(item)
        return list(top_k_users_items)      
        
    def get_accuracy_score(self, user_recitems):
        count_2days = count_4days = count_10days = 0
        for user in user_recitems:
            recommended_items = set(user_recitems.get(user,[]))
            new_items_2days = set(self.test2days_user_item.get(user,[]))
            new_items_4days = set(self.test4days_user_item.get(user,[]))
            new_items_10days = set(self.test10days_user_item.get(user,[]))
            if len(recommended_items & new_items_2days) > 0:
                count_2days += 1
            if len(recommended_items & new_items_4days) > 0:
                count_4days += 1
            if len(recommended_items & new_items_10days) > 0:
                count_10days += 1
        print("For a 2-day testing period:", count_2days/10000)
        print("For a 4-day testing period:", count_4days/10000)
        print("For a 10-day testing period:", count_10days/10000)
        
    def make_item_recommendations(self):
        with open(dirpath + "/rank_item_quantity_price_sales.txt") as itemfile:  # pull the ranked items, and create clusters
            for line in itemfile:
                toks = line.strip().split("\t")
                if len(toks) == 5:
                    rank = int(toks[0])
                    if rank <= 40:
                        self.star_items.append(toks[1])
                    else:
                        self.tail_items.append(toks[1])
        #print(self.star_items)
        #print(self.tail_items)
        print("\nMaking Recommendations\n")
        print("\nAccuracy for Star Items\n")
        self.make_user_recommendations_selected_cluster(cluster_items = self.star_items, flag = 'S')
        print("\nAccuracy for Long Tail Items\n")
        self.make_user_recommendations_selected_cluster(cluster_items = self.tail_items, flag = 'T')
               
dirpath = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Files'
outputfilename1 = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Latent Vector Representation/Gaming_output_metapath_UGIGU_w50_l20.txt'
outputfilename2 = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/LINE/LINE_vec_2nd_wo_norm.txt'


def main():
    hrs = HybridRecSys()
    hrs.read_data(dirpath)
    hrs.read_meta2vec_output(outputfilename1)
    hrs.read_LINE_output(outputfilename2)
    hrs.make_item_recommendations()

if __name__ == "__main__":
    main()

#users 1496096
#games 22
#items 3825
No of users who bought items in first 2 days of test period: 2700
No of users who bought items in first 4 days of test period: 4999
No of users who bought items in first 10 days of test period: 10302

Making Recommendations


Accuracy for Star Items


Making Recommendations

#Number of users randomly selected: 10000


Accuracy for Hybrid Recommender System based on Top-5 similar users
For a 2-day testing period: 0.0061
For a 4-day testing period: 0.0132
For a 10-day testing period: 0.0275


Accuracy for Hybrid Recommender System based on Top-10 similar users
For a 2-day testing period: 0.0072
For a 4-day testing period: 0.0154
For a 10-day testing period: 0.0325


Accuracy for Hybrid Recommender System based on Top-15 similar users
For a 2-day testing period: 0.0077
For a 4-day testing period: 0.0162
For a 10-day testing period: 0.0342


Accuracy for Hybrid Recommender System based on Top-20 similar users
For a 2-day testing period: 0.008
For a 4-da

In [1]:
# Generic RecSys - Equal Weighted Hybrid RecSys
import sys
import os
import random
import numpy as np
from collections import Counter

class EqualWeightedHybridRecSys:
    def __init__(self):
        self.id_user = dict()
        self.user_id = dict()
        self.id_game = dict()
        self.game_id = dict()
        self.id_item = dict()
        self.item_id = dict()
        self.item_user = dict()
        self.user_item = dict()
        self.game_user = dict()
        self.user_game = dict()
        self.game_item = dict()
        self.item_game = dict()
        self.test2days_item_user = dict()
        self.test2days_user_item = dict()
        self.test4days_item_user = dict()
        self.test4days_user_item = dict()
        self.test10days_item_user = dict()
        self.test10days_user_item = dict()
        self.user_friend = dict()
        self.group_user = dict()
        self.user_group = dict()
        self.userlist = []
        self.selected_user_items = dict()

    def read_data(self, dirpath):
        with open(dirpath + "/id_user.txt") as udictfile:
            for line in udictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_user[toks[0]] = toks[1].replace(" ", "")    # {uid1: User1, uid2: User2....}
                    self.user_id[toks[1]] = toks[0].replace(" ", "")    # {User1: uid1, User2: uid2....}

        print("#users", len(self.id_user))

        with open(dirpath + "/id_game.txt") as gdictfile:
            for line in gdictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_game[toks[0]] = toks[1].replace(" ", "")    # {33302: LoLsg, 33305: LoLmy....}
                    self.game_id[toks[1]] = toks[0].replace(" ", "")    # {LoLsg: 33302, LoLmy: 33305....}

        print("#games", len(self.id_game))
        
        with open(dirpath + "/id_item.txt") as idictfile:
            for line in idictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_item[toks[0]] = toks[1].replace(" ", "")    # {itemid1: Item1, itemid2: Item2....}
                    self.item_id[toks[1]] = toks[0].replace(" ", "")    # {Item1: itemid1, Item2: itemid2....}

        print("#items", len(self.id_item)) # comprehensive list of items

        with open(dirpath + "/game_user.txt") as gufile:  # data pulled from user game activity for 20 days
            for line in gufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    g, u = toks[0], toks[1]
                    if g not in self.game_user:
                        self.game_user[g] = []
                    self.game_user[g].append(u)   # {gameid1: [uid1,uid2,...], gameid2: [uid2,uid3,....]} = dict of list of users playing games
                    if u not in self.user_game:
                        self.user_game[u] = []
                    self.user_game[u].append(g)   # {uid1: [gameid1,gameid2,...], uid2: [gameid2,gameid3,....]} = dict of list of games played by each user
        
        
        with open(dirpath + "/item_user.txt") as iufile:  # list of all items purchased by users
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.item_user:
                        self.item_user[i] = []
                    self.item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.user_item:
                        self.user_item[u] = []
                    self.user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        
        
        with open(dirpath + "/item_game.txt") as igfile:  # list of all in-game items
            for line in igfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, g = toks[0], toks[1]
                    if i not in self.item_game:
                        self.item_game[i] = []  
                    self.item_game[i].append(g) # {itemid1: [gameid1,gameid2,...], itemid2: [gameid2,gameid3,....]} = list of items that got recommended. Every item can be in multiple games
                    if g not in self.game_item:
                        self.game_item[g] = []
                    self.game_item[g].append(i) # {gameid1: [itemid1,itemid2,...], gameid2: [itemd2,itemd3,....]} = list of items available in-game for purchase
        
        with open(dirpath + "/test2days_item_user.txt") as iufile:  # list of all items purchased by users in 2 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test2days_item_user:
                        self.test2days_item_user[i] = []
                    self.test2days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test2days_user_item:
                        self.test2days_user_item[u] = []
                    self.test2days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 2 days of test period:", len(self.test2days_user_item))
        
        with open(dirpath + "/test4days_item_user.txt") as iufile:  # list of all items purchased by users in 4 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test4days_item_user:
                        self.test4days_item_user[i] = []
                    self.test4days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test4days_user_item:
                        self.test4days_user_item[u] = []
                    self.test4days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 4 days of test period:", len(self.test4days_user_item))
        
        with open(dirpath + "/test10days_item_user.txt") as iufile:  # list of all items purchased by users in 10 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test10days_item_user:
                        self.test10days_item_user[i] = []
                    self.test10days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test10days_user_item:
                        self.test10days_user_item[u] = []
                    self.test10days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 10 days of test period:", len(self.test10days_user_item))
        
        with open(dirpath + "/selected_user.txt") as sufile:  # pull the randomly selected users
            for user in sufile:
                self.userlist.append(user.strip())
                
        with open(dirpath + "/group_user.txt") as gufile:  # list of all in-game groups
            for line in gufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    g, u = toks[0], toks[1]
                    if g not in self.group_user:
                        self.group_user[g] = []
                    self.group_user[g].append(u)   # {groupid1: [uid1,uid2,...], groupid2: [uid2,uid3,....]} = dict of each group and its members
                    if u not in self.user_group:
                        self.user_group[u] = []
                    self.user_group[u].append(g)   # {uid1: [groupid1,groupid2,...], uid2: [groupid2,groupid3,....]} = dict of list of groups for each user
                
        with open(dirpath + "/user_friend.txt") as uffile:  # list of all friendship linkages
            for line in uffile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    u, f = toks[0], toks[1]
                    if u not in self.user_friend:
                        self.user_friend[u] = []
                    self.user_friend[u].append(f)   # {uid1: [friendid1,friendid2,...], uid2: [friendid2,friendid3,....]} = dict of each person's friends
        
    def read_meta2vec_output(self, outputfilename):
        self.user_vec = dict()
        self.game_vec = dict()
        self.item_vec = dict()
        with open(outputfilename) as nvfile:             
            for line in nvfile:
                toks = line.strip().split(" ")
                if len(toks) == 129:
                    node = toks[0]
                    del toks[0]
                    if node[0] == 'u': 
                        self.user_vec[self.user_id[node]] = toks    # {uid1: [user_vector_values], uid2: [user_vector_values]..}
                    elif node[0] == 'g':
                        self.game_vec[self.game_id[node]] = toks    # {gameid1: [game_vector_values], gameid2: [game_vector_values]..}
                    elif node[0] == 'i':
                        self.item_vec[self.item_id[node]] = toks    # {itemid1: [item_vector_values], itemid2: [item_vector_values]..}
        
    def read_LINE_output(self, outputfilename):
        self.user_line_vec = dict()
        with open(outputfilename) as nvfile:             
            for line in nvfile:
                toks = line.strip().split(" ")
                if len(toks) == 129:
                    node = toks[0]
                    del toks[0] 
                    self.user_line_vec[node] = toks    # {uid1: [user_vector_values], uid2: [user_vector_values]..}
        
    def get_cosine_similarity(self, vec1, vec2):
        a = np.array(vec1, dtype=float)
        b = np.array(vec2, dtype=float)
        dot_product = np.dot(a, b)
        norm_a = np.linalg.norm(a)
        norm_b = np.linalg.norm(b)
        return dot_product / (norm_a * norm_b)
    
    def make_item_recommendations_selected_users(self):
        top_5_user_recitems = dict()
        top_10_user_recitems = dict()
        top_15_user_recitems = dict()
        top_20_user_recitems = dict()
        default_weight_meta = 0.2
        default_weight_line = 0.2
        default_weight_friends = 0.2
        default_weight_community = 0.2
        default_weight_user_item_cf = 0.2
        print("\nMaking Recommendations\n")
        print("#Number of users randomly selected:", len(self.userlist))
        for user1 in self.userlist:                                                                                
            user1_metavec = self.user_vec.get(user1,[])                                                            
            user1_linevec = self.user_line_vec.get(user1,[])
            friends1 = set(self.user_friend.get(user1,[]))
            groups1 = set(self.user_group.get(user1,[]))
            items1 = set(self.user_item.get(user1,[]))
            self.user_simscore = {}
            self.user_rec = {}
            for user2 in self.user_item: 
                user2_metavec = self.user_vec.get(user2,[])                                                            
                user2_linevec = self.user_line_vec.get(user2,[])
                friends2 = set(self.user_friend.get(user2,[]))
                groups2 = set(self.user_group.get(user2,[]))
                items2 = set(self.user_item.get(user2,[]))
                if (user1 == user2): continue
                mutual_friends = len(friends1 & friends2)
                total_friends = len(friends1 | friends2)
                mutual_groups = len(groups1 & groups2)
                total_groups = len(groups1 | groups2)
                mutual_items = len(items1 & items2)
                total_items = len(items1 | items2)
                if (total_friends > 0):
                    friend_similarity_score = mutual_friends/total_friends
                else:
                    friend_similarity_score = 0
                if (total_groups > 0):
                    group_similarity_score = mutual_groups/total_groups
                else:
                    group_similarity_score = 0
                if (total_items > 0):
                    item_cf_similarity_score = mutual_items/total_items
                else:
                    item_cf_similarity_score = 0
                if (user1_metavec != []) and (user2_metavec != []):
                    meta_cosine_sim_score = round(self.get_cosine_similarity(user1_metavec, user2_metavec), 3)
                else:
                    meta_cosine_sim_score = 0
                if (user1_linevec != []) and (user2_linevec != []):
                    line_cosine_sim_score = round(self.get_cosine_similarity(user1_linevec, user2_linevec), 3)
                else:
                    line_cosine_sim_score = 0
                weighted_sim_score = (meta_cosine_sim_score * default_weight_meta) + (line_cosine_sim_score * default_weight_line) + (friend_similarity_score * default_weight_friends) + (group_similarity_score * default_weight_community) + (item_cf_similarity_score * default_weight_user_item_cf)
                rec_items = [item for item in list(items2) if item not in list(items1)]                                
                self.user_rec[user2] = rec_items                    
                self.user_simscore[user2] = weighted_sim_score
            self.sorted_user_simscore = sorted(self.user_simscore.items(), key=lambda x: x[1], reverse = True)
            top_5_user_recitems[user1] = self.get_top_k_user_items(k=5)
            top_10_user_recitems[user1] = self.get_top_k_user_items(k=10)
            top_15_user_recitems[user1] = self.get_top_k_user_items(k=15)
            top_20_user_recitems[user1] = self.get_top_k_user_items(k=20)
        print("\n")
        print("Accuracy for Equal-Weighted Hybrid Recommender System based on Top-5 similar users")
        self.get_accuracy_score(user_recitems = top_5_user_recitems)
        print("\n")
        print("Accuracy for Equal-Weighted Hybrid Recommender System based on Top-10 similar users")
        self.get_accuracy_score(user_recitems = top_10_user_recitems)
        print("\n")
        print("Accuracy for Equal-Weighted Hybrid Recommender System based on Top-15 similar users")
        self.get_accuracy_score(user_recitems = top_15_user_recitems)  
        print("\n")
        print("Accuracy for Equal-Weighted Hybrid Recommender System based on Top-20 similar users")
        self.get_accuracy_score(user_recitems = top_20_user_recitems)
        
    def get_top_k_user_items(self, k):
        top_k_users = [user for user,_ in self.sorted_user_simscore[0:k]]   # getting the top k most similar users to the focal user
        top_k_users_items = set()                                           # now we fetch the recommended items for these top users
        for user in top_k_users:
            items = self.user_rec[user]
            for item in items:
                top_k_users_items.add(item)
        return list(top_k_users_items)      
        
    def get_accuracy_score(self, user_recitems):
        count_2days = count_4days = count_10days = 0
        for user in user_recitems:
            recommended_items = set(user_recitems.get(user,[]))
            new_items_2days = set(self.test2days_user_item.get(user,[]))
            new_items_4days = set(self.test4days_user_item.get(user,[]))
            new_items_10days = set(self.test10days_user_item.get(user,[]))
            if len(recommended_items & new_items_2days) > 0:
                count_2days += 1
            if len(recommended_items & new_items_4days) > 0:
                count_4days += 1
            if len(recommended_items & new_items_10days) > 0:
                count_10days += 1
        print("For a 2-day testing period:", count_2days/10000)
        print("For a 4-day testing period:", count_4days/10000)
        print("For a 10-day testing period:", count_10days/10000)
                            
                
dirpath = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Files'
outputfilename1 = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Latent Vector Representation/Gaming_output_metapath_UGIGU_w50_l20.txt'
outputfilename2 = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/LINE/LINE_vec_2nd_wo_norm.txt'


def main():
    ewhrs = EqualWeightedHybridRecSys()
    ewhrs.read_data(dirpath)
    ewhrs.read_meta2vec_output(outputfilename1)
    ewhrs.read_LINE_output(outputfilename2)
    ewhrs.make_item_recommendations_selected_users()

if __name__ == "__main__":
    main()

#users 1496096
#games 22
#items 3825
No of users who bought items in first 2 days of test period: 2700
No of users who bought items in first 4 days of test period: 4999
No of users who bought items in first 10 days of test period: 10302

Making Recommendations

#Number of users randomly selected: 10000


Accuracy for Equal-Weighted Hybrid Recommender System based on Top-5 similar users
For a 2-day testing period: 0.0048
For a 4-day testing period: 0.0096
For a 10-day testing period: 0.0206


Accuracy for Equal-Weighted Hybrid Recommender System based on Top-10 similar users
For a 2-day testing period: 0.0061
For a 4-day testing period: 0.0115
For a 10-day testing period: 0.025


Accuracy for Equal-Weighted Hybrid Recommender System based on Top-15 similar users
For a 2-day testing period: 0.0064
For a 4-day testing period: 0.0122
For a 10-day testing period: 0.0267


Accuracy for Equal-Weighted Hybrid Recommender System based on Top-20 similar users
For a 2-day testing period: 0.0066
F

In [2]:
# Long Tail Analysis - Equal Weighted Hybrid RecSys
import sys
import os
import random
import numpy as np
from collections import Counter

class EqualWeightedHybridRecSys:
    def __init__(self):
        self.id_user = dict()
        self.user_id = dict()
        self.id_game = dict()
        self.game_id = dict()
        self.id_item = dict()
        self.item_id = dict()
        self.item_user = dict()
        self.user_item = dict()
        self.game_user = dict()
        self.user_game = dict()
        self.game_item = dict()
        self.item_game = dict()
        self.test2days_item_user = dict()
        self.test2days_user_item = dict()
        self.test4days_item_user = dict()
        self.test4days_user_item = dict()
        self.test10days_item_user = dict()
        self.test10days_user_item = dict()
        self.user_friend = dict()
        self.group_user = dict()
        self.user_group = dict()
        self.userlist = []
        self.selected_user_items = dict()
        self.star_items = []
        self.tail_items = []
        self.all_items = []

    def read_data(self, dirpath):
        with open(dirpath + "/id_user.txt") as udictfile:
            for line in udictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_user[toks[0]] = toks[1].replace(" ", "")    # {uid1: User1, uid2: User2....}
                    self.user_id[toks[1]] = toks[0].replace(" ", "")    # {User1: uid1, User2: uid2....}

        print("#users", len(self.id_user))

        with open(dirpath + "/id_game.txt") as gdictfile:
            for line in gdictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_game[toks[0]] = toks[1].replace(" ", "")    # {33302: LoLsg, 33305: LoLmy....}
                    self.game_id[toks[1]] = toks[0].replace(" ", "")    # {LoLsg: 33302, LoLmy: 33305....}

        print("#games", len(self.id_game))
        
        with open(dirpath + "/id_item.txt") as idictfile:
            for line in idictfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    self.id_item[toks[0]] = toks[1].replace(" ", "")    # {itemid1: Item1, itemid2: Item2....}
                    self.item_id[toks[1]] = toks[0].replace(" ", "")    # {Item1: itemid1, Item2: itemid2....}

        print("#items", len(self.id_item)) # comprehensive list of items

        with open(dirpath + "/game_user.txt") as gufile:  # data pulled from user game activity for 20 days
            for line in gufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    g, u = toks[0], toks[1]
                    if g not in self.game_user:
                        self.game_user[g] = []
                    self.game_user[g].append(u)   # {gameid1: [uid1,uid2,...], gameid2: [uid2,uid3,....]} = dict of list of users playing games
                    if u not in self.user_game:
                        self.user_game[u] = []
                    self.user_game[u].append(g)   # {uid1: [gameid1,gameid2,...], uid2: [gameid2,gameid3,....]} = dict of list of games played by each user
        
        
        with open(dirpath + "/item_user.txt") as iufile:  # list of all items purchased by users
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.item_user:
                        self.item_user[i] = []
                    self.item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.user_item:
                        self.user_item[u] = []
                    self.user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        
        
        with open(dirpath + "/item_game.txt") as igfile:  # list of all in-game items
            for line in igfile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, g = toks[0], toks[1]
                    if i not in self.item_game:
                        self.item_game[i] = []  
                    self.item_game[i].append(g) # {itemid1: [gameid1,gameid2,...], itemid2: [gameid2,gameid3,....]} = list of items that got recommended. Every item can be in multiple games
                    if g not in self.game_item:
                        self.game_item[g] = []
                    self.game_item[g].append(i) # {gameid1: [itemid1,itemid2,...], gameid2: [itemd2,itemd3,....]} = list of items available in-game for purchase
        
        with open(dirpath + "/test2days_item_user.txt") as iufile:  # list of all items purchased by users in 2 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test2days_item_user:
                        self.test2days_item_user[i] = []
                    self.test2days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test2days_user_item:
                        self.test2days_user_item[u] = []
                    self.test2days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 2 days of test period:", len(self.test2days_user_item))
        
        with open(dirpath + "/test4days_item_user.txt") as iufile:  # list of all items purchased by users in 4 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test4days_item_user:
                        self.test4days_item_user[i] = []
                    self.test4days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test4days_user_item:
                        self.test4days_user_item[u] = []
                    self.test4days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 4 days of test period:", len(self.test4days_user_item))
        
        with open(dirpath + "/test10days_item_user.txt") as iufile:  # list of all items purchased by users in 10 days of testing period
            for line in iufile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    i, u = toks[0], toks[1]
                    if i not in self.test10days_item_user:
                        self.test10days_item_user[i] = []
                    self.test10days_item_user[i].append(u)   # {itemid1: [uid1,uid2,...], itemid2: [uid2,uid3,....]} = dict of list of users purchasing each item 
                    if u not in self.test10days_user_item:
                        self.test10days_user_item[u] = []
                    self.test10days_user_item[u].append(i)   # {uid1: [itemid1,itemid2,...], uid2: [itemd2,itemd3,....]} = dict of list of items purchased by each user
        print("No of users who bought items in first 10 days of test period:", len(self.test10days_user_item))
        
        with open(dirpath + "/selected_user.txt") as sufile:  # pull the randomly selected users
            for user in sufile:
                self.userlist.append(user.strip())
                
        with open(dirpath + "/user_friend.txt") as uffile:  # list of all friendship linkages
            for line in uffile:
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    u, f = toks[0], toks[1]
                    if u not in self.user_friend:
                        self.user_friend[u] = []
                    self.user_friend[u].append(f)   # {uid1: [friendid1,friendid2,...], uid2: [friendid2,friendid3,....]} = dict of each person's friends
        
    def read_meta2vec_output(self, outputfilename):
        self.user_vec = dict()
        self.game_vec = dict()
        self.item_vec = dict()
        with open(outputfilename) as nvfile:             
            for line in nvfile:
                toks = line.strip().split(" ")
                if len(toks) == 129:
                    node = toks[0]
                    del toks[0]
                    if node[0] == 'u': 
                        self.user_vec[self.user_id[node]] = toks    # {uid1: [user_vector_values], uid2: [user_vector_values]..}
                    elif node[0] == 'g':
                        self.game_vec[self.game_id[node]] = toks    # {gameid1: [game_vector_values], gameid2: [game_vector_values]..}
                    elif node[0] == 'i':
                        self.item_vec[self.item_id[node]] = toks    # {itemid1: [item_vector_values], itemid2: [item_vector_values]..}
        
    def read_LINE_output(self, outputfilename):
        self.user_line_vec = dict()
        with open(outputfilename) as nvfile:             
            for line in nvfile:
                toks = line.strip().split(" ")
                if len(toks) == 129:
                    node = toks[0]
                    del toks[0] 
                    self.user_line_vec[node] = toks    # {uid1: [user_vector_values], uid2: [user_vector_values]..}
        
    def get_cosine_similarity(self, vec1, vec2):
        a = np.array(vec1, dtype=float)
        b = np.array(vec2, dtype=float)
        dot_product = np.dot(a, b)
        norm_a = np.linalg.norm(a)
        norm_b = np.linalg.norm(b)
        return dot_product / (norm_a * norm_b)
    
    def make_user_recommendations_selected_cluster(self, cluster_items, flag):
        top_5_user_recitems = dict()
        top_10_user_recitems = dict()
        top_15_user_recitems = dict()
        top_20_user_recitems = dict()
        default_weight_meta = 0.2
        default_weight_line = 0.2
        default_weight_friends = 0.2
        default_weight_community = 0.2
        default_weight_user_item_cf = 0.2
        print("\nMaking Recommendations\n")
        print("#Number of users randomly selected:", len(self.userlist))
        for user1 in self.userlist:                                                                                
            user1_metavec = self.user_vec.get(user1,[])                                                            
            user1_linevec = self.user_line_vec.get(user1,[])
            friends1 = set(self.user_friend.get(user1,[]))
            groups1 = set(self.user_group.get(user1,[]))
            items1 = set(self.user_item.get(user1,[]))
            self.user_simscore = {}
            self.user_rec = {}
            for user2 in self.user_item: 
                user2_metavec = self.user_vec.get(user2,[])                                                            
                user2_linevec = self.user_line_vec.get(user2,[])
                friends2 = set(self.user_friend.get(user2,[]))
                groups2 = set(self.user_group.get(user2,[]))
                items2 = set(self.user_item.get(user2,[]))
                if (user1 == user2): continue
                rec_items = []
                mutual_friends = len(friends1 & friends2)
                total_friends = len(friends1 | friends2)
                mutual_groups = len(groups1 & groups2)
                total_groups = len(groups1 | groups2)
                mutual_items = len(items1 & items2)
                total_items = len(items1 | items2)
                if (total_friends > 0):
                    friend_similarity_score = mutual_friends/total_friends
                else:
                    friend_similarity_score = 0
                if (total_groups > 0):
                    group_similarity_score = mutual_groups/total_groups
                else:
                    group_similarity_score = 0
                if (total_items > 0):
                    item_cf_similarity_score = mutual_items/total_items
                else:
                    item_cf_similarity_score = 0
                if (user1_metavec != []) and (user2_metavec != []):
                    meta_cosine_sim_score = round(self.get_cosine_similarity(user1_metavec, user2_metavec), 3)
                else:
                    meta_cosine_sim_score = 0
                if (user1_linevec != []) and (user2_linevec != []):
                    line_cosine_sim_score = round(self.get_cosine_similarity(user1_linevec, user2_linevec), 3)
                else:
                    line_cosine_sim_score = 0
                weighted_sim_score = (meta_cosine_sim_score * default_weight_meta) + (line_cosine_sim_score * default_weight_line) + (friend_similarity_score * default_weight_friends) + (group_similarity_score * default_weight_community) + (item_cf_similarity_score * default_weight_user_item_cf)
                for item in cluster_items:
                    if item not in list(items1):
                        if item in list(items2):
                            rec_items.append(item)
                self.user_rec[user2] = rec_items                    
                self.user_simscore[user2] = weighted_sim_score       
            self.sorted_user_simscore = sorted(self.user_simscore.items(), key=lambda x: x[1], reverse = True)
            top_5_user_recitems[user1] = self.get_top_k_user_items(k=5)
            top_10_user_recitems[user1] = self.get_top_k_user_items(k=10)
            top_15_user_recitems[user1] = self.get_top_k_user_items(k=15)
            top_20_user_recitems[user1] = self.get_top_k_user_items(k=20)
        print("\n")
        print("Accuracy for Equal-Weighted Hybrid Recommender System based on Top-5 similar users")
        self.get_accuracy_score(user_recitems = top_5_user_recitems)
        print("\n")
        print("Accuracy for Equal-Weighted Hybrid Recommender System based on Top-10 similar users")
        self.get_accuracy_score(user_recitems = top_10_user_recitems)
        print("\n")
        print("Accuracy for Equal-Weighted Hybrid Recommender System based on Top-15 similar users")
        self.get_accuracy_score(user_recitems = top_15_user_recitems)  
        print("\n")
        print("Accuracy for Equal-Weighted Hybrid Recommender System based on Top-20 similar users")
        self.get_accuracy_score(user_recitems = top_20_user_recitems)
        
    def get_top_k_user_items(self, k):
        top_k_users = [user for user,_ in self.sorted_user_simscore[0:k]]   # getting the top k most similar users to the focal user
        top_k_users_items = set()                                           # now we fetch the recommended items for these top users
        for user in top_k_users:
            items = self.user_rec[user]
            for item in items:
                top_k_users_items.add(item)
        return list(top_k_users_items)      
        
    def get_accuracy_score(self, user_recitems):
        count_2days = count_4days = count_10days = 0
        for user in user_recitems:
            recommended_items = set(user_recitems.get(user,[]))
            new_items_2days = set(self.test2days_user_item.get(user,[]))
            new_items_4days = set(self.test4days_user_item.get(user,[]))
            new_items_10days = set(self.test10days_user_item.get(user,[]))
            if len(recommended_items & new_items_2days) > 0:
                count_2days += 1
            if len(recommended_items & new_items_4days) > 0:
                count_4days += 1
            if len(recommended_items & new_items_10days) > 0:
                count_10days += 1
        print("For a 2-day testing period:", count_2days/10000)
        print("For a 4-day testing period:", count_4days/10000)
        print("For a 10-day testing period:", count_10days/10000)
        
    def make_item_recommendations(self):
        with open(dirpath + "/rank_item_quantity_price_sales.txt") as itemfile:  # pull the ranked items, and create clusters
            for line in itemfile:
                toks = line.strip().split("\t")
                if len(toks) == 5:
                    rank = int(toks[0])
                    if rank <= 40:
                        self.star_items.append(toks[1])
                    else:
                        self.tail_items.append(toks[1])
        #print(self.star_items)
        #print(self.tail_items)
        print("\nMaking Recommendations\n")
        print("\nAccuracy for Star Items\n")
        self.make_user_recommendations_selected_cluster(cluster_items = self.star_items, flag = 'S')
        print("\nAccuracy for Long Tail Items\n")
        self.make_user_recommendations_selected_cluster(cluster_items = self.tail_items, flag = 'T')
               
dirpath = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Files'
outputfilename1 = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/Latent Vector Representation/Gaming_output_metapath_UGIGU_w50_l20.txt'
outputfilename2 = 'C:/Vidit/PhD/RA Work/KZ - RA/Gaming Project/data/LINE/LINE_vec_2nd_wo_norm.txt'


def main():
    ewhrs = EqualWeightedHybridRecSys()
    ewhrs.read_data(dirpath)
    ewhrs.read_meta2vec_output(outputfilename1)
    ewhrs.read_LINE_output(outputfilename2)
    ewhrs.make_item_recommendations()

if __name__ == "__main__":
    main()

#users 1496096
#games 22
#items 3825
No of users who bought items in first 2 days of test period: 2700
No of users who bought items in first 4 days of test period: 4999
No of users who bought items in first 10 days of test period: 10302

Making Recommendations


Accuracy for Star Items


Making Recommendations

#Number of users randomly selected: 10000


Accuracy for Equal-Weighted Hybrid Recommender System based on Top-5 similar users
For a 2-day testing period: 0.0044
For a 4-day testing period: 0.009
For a 10-day testing period: 0.0194


Accuracy for Equal-Weighted Hybrid Recommender System based on Top-10 similar users
For a 2-day testing period: 0.0053
For a 4-day testing period: 0.0106
For a 10-day testing period: 0.0231


Accuracy for Equal-Weighted Hybrid Recommender System based on Top-15 similar users
For a 2-day testing period: 0.0055
For a 4-day testing period: 0.0112
For a 10-day testing period: 0.0247


Accuracy for Equal-Weighted Hybrid Recommender System based on Top-20